In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score
import time

2023-06-02 13:48:33.798255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the dataset
data_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/'
dataset = np.load(data_path + 'PCA_dataset1.npy', allow_pickle=True)[()]

x_train = dataset['train_stars_pca']
x_val = dataset['validation_stars_pca']
x_test = dataset['test_stars_pca']
y_train = dataset['train_C']
y_val = dataset['validation_C']
y_test = dataset['test_C']
SED_test = dataset['test_SEDs']


In [3]:
# Hyperparameters
PCA_components = 24
model_learning_rate = 0.05
N_epochs = 25
N_committee = 48

In [4]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate

In [5]:
# Define the model architecture
def create_model():
    initializer = tf.keras.initializers.GlorotNormal(seed = 1)
    model = tf.keras.Sequential([
        layers.Dense(26, input_shape=[PCA_components], activation='sigmoid', kernel_initializer= initializer),
        layers.Dense(26, activation='sigmoid', kernel_initializer= initializer),
        layers.Dense(1, activation = 'linear', kernel_initializer= initializer)
    ])
    model.compile(
        loss = tf.keras.losses.MeanSquaredError(),
        optimizer = tf.keras.optimizers.Adam(learning_rate = model_learning_rate)
    )
    return model

In [6]:

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights=True)
progbar_logger = tf.keras.callbacks.ProgbarLogger()

start_time = time.time() # Measure execution time

# Train a committee of 48 neural networks and make predictions
committee = []
committee_predictions = []
for i in range(N_committee):
    model = create_model()
    model.fit(x_train, y_train, epochs= N_epochs, callbacks = [progbar_logger,early_stopping], validation_data=(x_val,y_val))
    committee.append(model)
    committee_predictions.append(model.predict(x_test, verbose = 0).reshape(-1)) # Predict the scalar parameter C using the committee    

# Calculate the execution time
end_time = time.time()
execution_time = end_time - start_time

committee_predictions = np.array(committee_predictions)
y_pred = np.mean(committee_predictions, axis=0)
pred_variance = np.var(committee_predictions, axis=0)
SED_pred = CtoSEDarray(y_pred,pred_variance)


Epoch 1/25
1/1 [==============================] - 1s 881ms/sample - loss: 22.5677 - val_loss: 10.5414
Epoch 2/25
1/1 [==============================] - 0s 33ms/sample - loss: 12.6492 - val_loss: 5.5453
Epoch 3/25
1/1 [==============================] - 0s 41ms/sample - loss: 6.9959 - val_loss: 3.5187
Epoch 4/25
1/1 [==============================] - 0s 31ms/sample - loss: 4.3988 - val_loss: 3.4937
Epoch 5/25
1/1 [==============================] - 0s 34ms/sample - loss: 3.8884 - val_loss: 4.5365
Epoch 6/25
1/1 [==============================] - 0s 44ms/sample - loss: 4.5584 - val_loss: 5.6329
Epoch 7/25
1/1 [==============================] - 0s 59ms/sample - loss: 5.4256 - val_loss: 6.1743
Epoch 8/25
1/1 [==============================] - 0s 42ms/sample - loss: 5.8742 - val_loss: 6.0871
Epoch 9/25
1/1 [==============================] - 0s 64ms/sample - loss: 5.8010 - val_loss: 5.5734
Epoch 1/25
1/1 [==============================] - 1s 683ms/sample - loss: 22.5677 - val_loss: 10.5414
Epo

In [7]:
# Evaluate the performance of the committee

mse = np.mean((y_test - y_pred)**2)
print('MSE:', mse)
print("Variance: ", pred_variance)

f1 = f1_score(SED_test, SED_pred, average = None)
print('\nF1 score:', f1)
print(np.mean(f1[:13]))

confusion_matrix = tf.math.confusion_matrix(SED_test, SED_pred) 
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)

MSE: 2.626908451965619
Variance:  [3.5716614e-12 9.0475777e-13 9.0949470e-13 3.6806114e-12 8.1854523e-12
 5.6843419e-12 1.1141310e-11 5.6843419e-14 3.4721854e-12 3.5385028e-12]

F1 score: [0.         0.         0.         0.18181818 0.         0.
 0.         0.        ]
0.022727272727272728

Confusion matrix:
tf.Tensor(
[[0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]], shape=(12, 12), dtype=int32)

Success rate: 0.2


In [8]:
#Save the committee models
import os

# Create a directory to save the models
os.makedirs("committee_models", exist_ok=True)

# Save the models
for i, model in enumerate(committee):
    model.save(f"committee_models/model_{i}.h5")